In [1]:
# import pandas for data manipulation
import pandas as pd
import numpy as np
from itertools import product

In [2]:
# Read in new dataset
sales = pd.read_csv('customer_data_latest.csv', encoding='utf-8')
sales = sales[sales.filter(regex='^(?!Unnamed)').columns]

# Remove unecessary 'SESSION_TYPE' and 'PROPOSITION_TYPE' columns
sales = sales.drop(columns=['SESSION_TYPE', 'PROPOSITION_TYPE'])

# Sort by 'DATE'
sales["DATE"] = pd.to_datetime(sales["DATE"])
sales = sales.sort_values(by="DATE")
sales.head(10)

,DATE,PROPOSITION,SESSION,SITE,SALES,CUST_ID,ACTIVE_SUB
342,2017-01-02,In-store,Night,9,89,76,1
890,2017-01-04,Delivery,Night,2,282,104,0
861,2017-01-05,Pick-up,Afternoon,2,182,127,1
496,2017-01-07,In-store,Afternoon,5,302,78,1
871,2017-01-07,Delivery,Night,2,297,103,0
788,2017-01-08,Pick-up,Morning,7,209,110,0
573,2017-01-11,Delivery,Morning,6,87,154,0
822,2017-01-11,In-store,Morning,4,136,102,1
718,2017-01-11,Pick-up,Morning,8,471,97,1
622,2017-01-12,In-store,Morning,7,265,162,1


In [3]:
# Pad out missing dates
def pad_missing_dates(df):
        
    # Get the start and end date of dataframe
    start_date = pd.to_datetime(df['DATE'].min())
    end_date = pd.to_datetime(df['DATE'].max())    
    dates = pd.date_range(start=start_date, end=end_date)
    
    # Get all unique combinations
    props = sales.PROPOSITION.unique()
    sessions = sales.SESSION.unique()
    sites = sales.SITE.unique()
    
    # Create skeleton dataframe with entire date range   
    skeleton = pd.DataFrame(list(product(dates, props, sessions, sites)), 
                                 columns=['DATE', 'PROPOSITION', 'SESSION', 'SITE'])
    skeleton.sort_values(['DATE', 'SITE', 'PROPOSITION', 'SESSION'])

    # Perform outer merge
    df = pd.merge(df, skeleton, how='outer', on=['DATE', 'PROPOSITION', 'SESSION', 'SITE'])
    df2 = df.sort_values(by=['DATE', 'PROPOSITION', 'SESSION', 'SITE']).reset_index(drop='True')
    
    # Fill in zeroes, change proposition, session to string types
    df2 = df2.fillna(0)
    return df2

In [4]:
padded = pad_missing_dates(sales)
padded.head()

,DATE,PROPOSITION,SESSION,SITE,SALES,CUST_ID,ACTIVE_SUB
0,2017-01-02,Delivery,Afternoon,1,0.0,0.0,0.0
1,2017-01-02,Delivery,Afternoon,2,0.0,0.0,0.0
2,2017-01-02,Delivery,Afternoon,3,0.0,0.0,0.0
3,2017-01-02,Delivery,Afternoon,4,0.0,0.0,0.0
4,2017-01-02,Delivery,Afternoon,5,0.0,0.0,0.0


In [5]:
# Default columns (and relevant functions) to aggregate on
DEFAULT_AGGREGATIONS = {'SALES': 'sum', 'CUST_ID': 'sum', 'ACTIVE_SUB': 'sum', 
                        'SALES_DELTA': 'sum', 'SUB_DELTA': 'sum'}

def group_on_variable(cols, df):
    """
    Helper method for grouping on and removing columns from
    the sales dataframe. We bake in the particular necessary aggregations 
    for our columns.
    """
    
    # Change CUST_ID to be '1' if customer, '0' if not
    df['CUST_ID'] = df['CUST_ID'].astype(bool)
    
    aggregation_functions = {
        col: f for (col, f) in DEFAULT_AGGREGATIONS.items() if col in df
    }

    df2 = df.groupby(cols, as_index=False).aggregate(aggregation_functions)
    
    # Rename CUST_ID to TOTAL_CUST and ACTIVE_SUB to TOTAL_ACTIVE
    df2.rename(columns={'CUST_ID': 'TOTAL_CUST', 'ACTIVE_SUB': 'TOTAL_ACTIVE'}, inplace=True)
    df2.sort_values(['DATE'])
    return df2

In [6]:
group_on_variable(['DATE'], padded).head(10)

,DATE,SALES,TOTAL_CUST,TOTAL_ACTIVE
0,2017-01-02,89.0,1,1.0
1,2017-01-03,0.0,0,0.0
2,2017-01-04,282.0,1,0.0
3,2017-01-05,182.0,1,1.0
4,2017-01-06,0.0,0,0.0
5,2017-01-07,599.0,2,1.0
6,2017-01-08,209.0,1,0.0
7,2017-01-09,0.0,0,0.0
8,2017-01-10,0.0,0,0.0
9,2017-01-11,694.0,3,2.0


In [7]:
def rolling_week(df, num_weeks, granularity):
    
    gap = num_weeks * 7
    
    # Do rolling sum
    current = df.shift(-gap).rolling(gap)['SALES'].sum()
    previous = df.rolling(gap)['SALES'].sum()
    df['DATA'] = (current/previous - 1) * 100
    
    # Shift data down
    df['WEEK_TREND'] = df.groupby(granularity)['DATA'].shift(gap)
    return df

In [8]:
def week_trend(sales, num_weeks, granularity):
    new_sales = group_on_variable(granularity, sales)
    granularity.remove('DATE')
    return new_sales.groupby(granularity, as_index=False).apply(rolling_week, num_weeks, granularity).drop(columns='DATA')

In [9]:
pd.set_option('display.max_rows', 1500)
result = week_trend(padded, 2, ['DATE', 'SITE'])
result.head(1000)

,DATE,SITE,SALES,TOTAL_CUST,TOTAL_ACTIVE,WEEK_TREND
0,2017-01-02,1,0.0,0,0.0,NaN
1,2017-01-02,2,0.0,0,0.0,NaN
2,2017-01-02,3,0.0,0,0.0,NaN
3,2017-01-02,4,0.0,0,0.0,NaN
4,2017-01-02,5,0.0,0,0.0,NaN
5,2017-01-02,6,0.0,0,0.0,NaN
6,2017-01-02,7,0.0,0,0.0,NaN
7,2017-01-02,8,0.0,0,0.0,NaN
8,2017-01-02,9,89.0,1,1.0,NaN
9,2017-01-02,10,0.0,0,0.0,NaN


In [11]:
def last_week(df, num_weeks, granularity):
    
    gap = num_weeks * 7
    
    df['LAST_N_WEEK_SALES'] = df.rolling(gap)['SALES'].sum()
    return df
    

In [14]:
def total_last_n_weeks(sales, num_weeks, granularity):
    new_sales = group_on_variable(granularity, sales)
    granularity.remove('DATE')
    return new_sales.groupby(granularity, as_index=False).apply(last_week, num_weeks, granularity)
    

In [18]:
total_last_n_weeks(padded, 2, ['DATE', 'PROPOSITION']).head(1000)

,DATE,PROPOSITION,SALES,TOTAL_CUST,TOTAL_ACTIVE,LAST_N_WEEK_SALES
0,2017-01-02,Delivery,0.0,0,0.0,NaN
1,2017-01-02,In-store,89.0,1,1.0,NaN
2,2017-01-02,Pick-up,0.0,0,0.0,NaN
3,2017-01-03,Delivery,0.0,0,0.0,NaN
4,2017-01-03,In-store,0.0,0,0.0,NaN
5,2017-01-03,Pick-up,0.0,0,0.0,NaN
6,2017-01-04,Delivery,282.0,1,0.0,NaN
7,2017-01-04,In-store,0.0,0,0.0,NaN
8,2017-01-04,Pick-up,0.0,0,0.0,NaN
9,2017-01-05,Delivery,0.0,0,0.0,NaN
